# Parabricks Hands-On Workshop

## Tutorial 1: Germline Workflow
This workflow will demonstrate how to perform secondary analysis for DNA sequencing data from next-generation sequencing. This is relevant for population sequencing, rare disease detection, common disease sequencing, etc.
- Sample: 30x whole-genome sequencing from a human individual, HC002.
- Analysis: 1) Read alignment, 2) Variant calling

To save time from downloading data, all necessary data has been put in TWCC Cloud Object Services (COS), including sample data and reference genome library.

#### Download Data

We will download the reference human genome, its index files, a known variants file, and 
a 30x whole-genome sequencing sample for germline analysis.

In [1]:
# The tar file is 9.3GB and, when extracted, an additional 14GB
!wget http://cos.twcc.ai/pbworkshop/hg38/parabricks_sample.tar.gz
!tar -xzvf parabricks_sample.tar.gz
!mv parabricks_sample/* .
!rm -r parabricks_sample

URL transformed to HTTPS due to an HSTS policy
--2025-09-03 12:17:31--  https://cos.twcc.ai/pbworkshop/hg38/parabricks_sample.tar.gz
Resolving cos.twcc.ai (cos.twcc.ai)... 203.145.219.21
Connecting to cos.twcc.ai (cos.twcc.ai)|203.145.219.21|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11767623602 (11G) [binary/octet-stream]
Saving to: ‘parabricks_sample.tar.gz.1’

parabricks_sample.t 100%[===================>]  10.96G   173MB/s    in 1m 41s  

2025-09-03 12:19:12 (112 MB/s) - ‘parabricks_sample.tar.gz.1’ saved [11767623602/11767623602]

parabricks_sample/
parabricks_sample/Data/
parabricks_sample/Data/markdup_input.bam
parabricks_sample/Data/sample_2.fq.gz
parabricks_sample/Data/sample_1.fq.gz
parabricks_sample/Data/single_ended.bam
parabricks_sample/Ref/
parabricks_sample/Ref/Homo_sapiens_assembly38.fasta.sa
parabricks_sample/Ref/Homo_sapiens_assembly38.known_indels.vcf.gz.tbi
parabricks_sample/Ref/Homo_sapiens_assembly38.dict
parabricks_sample/Ref/Homo_s

In [13]:
!mv Data germline_data
!ls germline_data

markdup_input.bam  sample_1.fq.gz  sample_2.fq.gz  single_ended.bam


In [3]:
!ls Ref

Homo_sapiens_assembly38.dict
Homo_sapiens_assembly38.fasta
Homo_sapiens_assembly38.fasta.amb
Homo_sapiens_assembly38.fasta.ann
Homo_sapiens_assembly38.fasta.bwt
Homo_sapiens_assembly38.fasta.fai
Homo_sapiens_assembly38.fasta.pac
Homo_sapiens_assembly38.fasta.sa
Homo_sapiens_assembly38.known_indels.vcf.gz
Homo_sapiens_assembly38.known_indels.vcf.gz.tbi
ctat_genome_lib_build_dir
gencode.v48.primary_assembly.annotation.gtf
genomelib
genomelib.tar.gz


Make a new directory to store the output from this tutorial.

In [4]:
!mkdir germline_output

mkdir: cannot create directory ‘germline_output’: File exists


#### GPU Monitoring

In [5]:
!nvidia-smi

Wed Sep  3 12:20:44 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.161.08             Driver Version: 535.161.08   CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-32GB           On  | 00000000:1B:00.0 Off |                    0 |
| N/A   27C    P0              42W / 300W |      0MiB / 32768MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# Run the command below in the terminal
### watch -n 0.5 nvidia-smi
#

### Alignment: FASTQ to BAM

The alignment and variant calling steps in the germline workflow can run separately with these commands. Again, `--bwa-nstreams` is set to `1` because we are limited to a single V100 GPU. With more GPU memory, you can run more streams (i.g., 16) and shorten the time for running.

In [6]:
!pbrun fq2bam \
      --ref Ref/Homo_sapiens_assembly38.fasta \
      --in-fq germline_data/sample_1.fq.gz germline_data/sample_2.fq.gz \
      --out-bam germline_output/fq2bam_output.bam \
      --num-gpus 1 \
      --bwa-nstreams 1 

Please visit https://docs.nvidia.com/clara/#parabricks for detailed documentation



[Parabricks Options Mesg]: Checking argument compatibility
[Parabricks Options Mesg]: Set --bwa-options="-K #" to produce compatible pair-ended results with previous versions of
fq2bam or BWA MEM.
[Parabricks Options Mesg]: Automatically generating ID prefix
[Parabricks Options Mesg]: Read group created for /home/yingja1227/germline_data/sample_1.fq.gz and
/home/yingja1227/germline_data/sample_2.fq.gz
[Parabricks Options Mesg]: @RG\tID:HK3TJBCX2.1\tLB:lib1\tPL:bar\tSM:sample\tPU:HK3TJBCX2.1
[PB Info 2025-Sep-03 12:20:44] ------------------------------------------------------------------------------
[PB Info 2025-Sep-03 12:20:44] ||                 Parabricks accelerated Genomics Pipeline                 ||
[PB Info 2025-Sep-03 12:20:44] ||                              Version 4.4.0-1                             ||
[PB Info 2025-Sep-03 12:20:44] ||                      GPU-PBBWA mem, Sorting Phase-I    

### Variant Calling: BAM to VCF

#### GATK Haplotypecaller

In [7]:
!pbrun haplotypecaller -h

Please visit https://docs.nvidia.com/clara/#parabricks for detailed documentation

usage: pbrun haplotypecaller <options>
Help: pbrun haplotypecaller -h

Run HaplotypeCaller to convert BAM/CRAM to VCF.

options:
  -h, --help            show this help message and exit

Input Output file options:
  Options for Input and Output files for this tool.

  --ref REF             Path to the reference file. (default: None)
  --in-bam IN_BAM       Path to the input BAM/CRAM file for variant calling.
                        The argument may also be a local folder containing
                        several BAM files. (default: None)
  --in-recal-file IN_RECAL_FILE
                        Path to the input BQSR report. (default: None)
  --interval-file INTERVAL_FILE
                        Path to an interval file in one of these formats:
                        Picard-style (.interval_list or .picard), GATK-style
                        (.list or .intervals), or BED file (.bed). This option
       

This command will generate `.vcf` file from `.bam` file that we produced in the previous step.
For population studies, you may want to generate `.gvcf` files to me merged for joint genotyping in later steps. Otherwise, you can delete the` --gvcf` option and output as `.vcf` file.

In [8]:
!pbrun haplotypecaller \
      --ref Ref/Homo_sapiens_assembly38.fasta \
      --in-bam germline_output/fq2bam_output.bam \
      --out-variants germline_output/variants_gatk.g.vcf \
      --num-gpus 1\
      --htvc-low-memory
#    --gvcf   #Use this option if you want to generate .gvcf file

Please visit https://docs.nvidia.com/clara/#parabricks for detailed documentation

/usr/local/parabricks/binaries/bin/htvc /home/yingja1227/Ref/Homo_sapiens_assembly38.fasta /home/yingja1227/germline_output/fq2bam_output.bam 1 -o /home/yingja1227/germline_output/variants_gatk.g.vcf -nt 5 --low-memory
[PB Info 2025-Sep-03 12:29:13] ------------------------------------------------------------------------------
[PB Info 2025-Sep-03 12:29:13] ||                 Parabricks accelerated Genomics Pipeline                 ||
[PB Info 2025-Sep-03 12:29:13] ||                              Version 4.4.0-1                             ||
[PB Info 2025-Sep-03 12:29:13] ||                         GPU-GATK4 HaplotypeCaller                        ||
[PB Info 2025-Sep-03 12:29:13] ------------------------------------------------------------------------------
[PB Info 2025-Sep-03 12:29:25] 0 /home/yingja1227/germline_output/fq2bam_output.bam/home/yingja1227/germline_output/variants_gatk.g.vcf
[PB Info 202

The key difference between a regular VCF and a GVCF is that the GVCF has records for all sites, whether there is a variant call there or not. The goal is to have every site represented in the file in order to do joint analysis of a cohort in subsequent steps. (https://gatk.broadinstitute.org/hc/en-us/articles/360035531812-GVCF-Genomic-Variant-Call-Format)

In [9]:
!ls -lh germline_output/

total 9.1G
-rw-r--r-- 1 yingja1227 ACD113150 4.5G Sep  3 12:29 fq2bam_output.bam
-rw-r--r-- 1 yingja1227 ACD113150 3.1M Sep  3 12:29 fq2bam_output.bam.bai
-rw-r--r-- 1 yingja1227 ACD113150  86K Sep  3 12:29 fq2bam_output_chrs.txt
-rw-r--r-- 1 yingja1227 ACD113150 4.5G Aug 29 13:57 germline_out.bam
-rw-r--r-- 1 yingja1227 ACD113150 3.1M Aug 29 13:57 germline_out.bam.bai
-rw-r--r-- 1 yingja1227 ACD113150  86K Aug 29 13:57 germline_out_chrs.txt
-rw-r--r-- 1 yingja1227 ACD113150 387K Aug 29 13:56 recal.table
-rw-r--r-- 1 yingja1227 ACD113150  22M Aug 29 14:10 variants.vcf
-rw-r--r-- 1 yingja1227 ACD113150  23M Sep  3 12:46 variants_gatk.g.vcf
-rw-r--r-- 1 yingja1227 ACD113150 168K Sep  3 12:46 variants_gatk.g.vcf.idx


#### DeepVariant
Beside the Haplotype Caller from GATK, another popular variant caller is DeepVariant, which uses AI methods for variant calling. Parabricks speeeds up DeepVariant and can be used for both short-read and long-read sequencing by choosing the right option.

In [16]:
!pbrun deepvariant -h

Please visit https://docs.nvidia.com/clara/#parabricks for detailed documentation

usage: pbrun deepvariant <options>
Help: pbrun deepvariant -h

Run DeepVariant to convert BAM/CRAM to VCF.

optional arguments:
  -h, --help            show this help message and exit

Input Output file options:
  Options for Input and Output files for this tool:

  --ref REF             Path to the reference file. (default: None)
  --in-bam IN_BAM       Path to the input BAM/CRAM file for variant calling.
                        (default: None)
  --interval-file INTERVAL_FILE
                        Path to a BED file (.bed) for selective access. This
                        option can be used multiple times. (default: None)
  --out-variants OUT_VARIANTS
                        Path of the vcf/g.vcf/g.vcf.gz file after variant
                        calling. (default: None)
  --pb-model-file PB_MODEL_FILE
                        Path to a non-default parabricks model file for
                        de

Default is to run variant calling on short reads like Illumina.

In [18]:
!pbrun deepvariant \
    --ref Ref/Homo_sapiens_assembly38.fasta \
    --in-bam germline_output/fq2bam_output.bam \
    --out-variants germline_output/variants_dv.vcf \
    --num-streams-per-gpu 2 \
    --run-partition \
    --gpu-num-per-partition 1 \
    --num-gpus 1

Please visit https://docs.nvidia.com/clara/#parabricks for detailed documentation

Detected 1 CUDA Capable device(s), considering 1 device(s)
  CUDA Driver Version / Runtime Version          12.2 / 12.0
Using model for CUDA Capability Major/Minor version number:    89
CUDA_VISIBLE_DEVICES=0 /usr/local/parabricks/binaries//bin/deepvariant /workdir/sample_data/Ref/Homo_sapiens_assembly38.fasta /workdir/outputdir/fq2bam_output.bam 1 2 -o 0.vcf  -L "chr1" -L "chr2" -L "chr3" -L "chr4" -L "chr5" -L "chr6" -L "chr7" -L "chrX" -L "chr8" -L "chr9" -L "chr11" -L "chr10" -L "chr12" -L "chr13" -L "chr14" -L "chr15" -L "chr16" -L "chr17" -L "chr18" -L "chr20" -L "chr19" -L "chrY" -L "chr22" -L "chr21" -L "chrM" -n 6 --model /usr/local/parabricks/binaries//model/80+/shortread/deepvariant.eng --channel_insert_size --pileup_image_width 221 --max_reads_per_partition 1500 --partition_size 1000 --vsc_min_count_snps 2 --vsc_min_count_indels 2 --vsc_min_fraction_snps 0.12 --min_mapping_quality 5 --min_bas

DeepVariant from Parabricks has the ability to use multiple streams on a GPU. The number of streams that can be used depends on the available resources. The default number of streams is set to two but can be increased up to a maximum of six to get better performance. This is something that has to be experimented with, before getting the optimal number on your system. (https://docs.nvidia.com/clara/parabricks/4.1.0/bestperformance.html#best-performance-for-deepvariant)

In [21]:
!pbrun deepvariant \
    --ref /workdir/sample_data/Ref/Homo_sapiens_assembly38.fasta \
    --in-bam /workdir/outputdir/fq2bam_output.bam \
    --out-variants /workdir/outputdir/variants_dv.vcf \
    --num-streams-per-gpu 4 \
    --run-partition \
    --gpu-num-per-partition 1 \
    --num-gpus 1

Please visit https://docs.nvidia.com/clara/#parabricks for detailed documentation

Detected 1 CUDA Capable device(s), considering 1 device(s)
  CUDA Driver Version / Runtime Version          12.2 / 12.0
Using model for CUDA Capability Major/Minor version number:    89
CUDA_VISIBLE_DEVICES=0 /usr/local/parabricks/binaries//bin/deepvariant /workdir/sample_data/Ref/Homo_sapiens_assembly38.fasta /workdir/outputdir/fq2bam_output.bam 1 4 -o 0.vcf  -L "chr1" -L "chr2" -L "chr3" -L "chr4" -L "chr5" -L "chr6" -L "chr7" -L "chrX" -L "chr8" -L "chr9" -L "chr11" -L "chr10" -L "chr12" -L "chr13" -L "chr14" -L "chr15" -L "chr16" -L "chr17" -L "chr18" -L "chr20" -L "chr19" -L "chrY" -L "chr22" -L "chr21" -L "chrM" -n 6 --model /usr/local/parabricks/binaries//model/80+/shortread/deepvariant.eng --channel_insert_size --pileup_image_width 221 --max_reads_per_partition 1500 --partition_size 1000 --vsc_min_count_snps 2 --vsc_min_count_indels 2 --vsc_min_fraction_snps 0.12 --min_mapping_quality 5 --min_bas

Using the --run-partition, --proposed-variants, and --gvcf options at the same time will lead to a substantial slowdown.

- gvcf

In [15]:
!pbrun deepvariant \
    --ref sample_data/Ref/Homo_sapiens_assembly38.fasta \
    --in-bam outputdir/germline_out.bam \
    --out-variants outputdir/variants_dv.g.vcf \
    --num-streams-per-gpu 1 \
    --gvcf \
    --num-gpus 1

Please visit https://docs.nvidia.com/clara/#parabricks for detailed documentation

Detected 1 CUDA Capable device(s), considering 1 device(s)
  CUDA Driver Version / Runtime Version          12.3 / 12.3
Using model for CUDA Capability Major/Minor version number:    70
/usr/local/parabricks/binaries/bin/deepvariant /tutorial/sample_data/Ref/Homo_sapiens_assembly38.fasta /tutorial/outputdir/germline_out.bam 1 1 -o /tutorial/outputdir/variants_dv.g.vcf -n 6 --model /usr/local/parabricks/binaries/model/70/shortread/deepvariant.eng -g --channel_insert_size --pileup_image_width 221 --max_reads_per_partition 1500 --partition_size 1000 --vsc_min_count_snps 2 --vsc_min_count_indels 2 --vsc_min_fraction_snps 0.12 --min_mapping_quality 5 --min_base_quality 10 --alt_aligned_pileup none --variant_caller VERY_SENSITIVE_CALLER --dbg_min_base_quality 15 --ws_min_windows_distance 80 --aux_fields_to_keep HP --p_error 0.001 --max_ins_size 10
[PB Info 2024-Oct-11 03:53:55] --------------------------------

### Run Germline Workflow: GATK (BWA-MEM + HaplotypeCaller)

This workflow runs the GATK germline pipeline in one command. This command will run read alignment followed by variant calling using HaplotypeCaller. It will not only output the VCF file, but also output intermediate BAM and BQSR files if needed. In this workshop, we are limited to a single V100 GPU with 16GB RAM. Therefore, the `--low-memory` option is used.

In [1]:
!pbrun germline -h

Please visit https://docs.nvidia.com/clara/#parabricks for detailed documentation


usage: pbrun germline <options>
Help: pbrun germline -h

Run Germline pipeline to convert FASTQ to VCF.

options:
  -h, --help            show this help message and exit

Input Output file options:
  Options for Input and Output files for this tool.

  --ref REF             Path to the reference file. (default: None)
  --in-fq [IN_FQ ...]   Path to the pair-ended FASTQ files followed by
                        optional read groups with quotes (Example:
                        "@RG\tID:foo\tLB:lib1\tPL:bar\tSM:sample\tPU:foo").
                        The files must be in fastq or fastq.gz format. All
                        sets of inputs should have a read group; otherwise,
                        none should have a read group, and it will be
                        automatically added by the pipeline. This option can
                        be repeated multiple times. Example 1: --in-fq
              

In [5]:
!pbrun germline \
    --ref Ref/Homo_sapiens_assembly38.fasta \
    --in-fq germline_data/sample_1.fq.gz germline_data/sample_2.fq.gz \
    --knownSites Ref/Homo_sapiens_assembly38.known_indels.vcf.gz \
    --out-bam germline_output/germline_out.bam \
    --out-variants germline_output/variants.vcf \
    --out-recal-file germline_output/recal.table \
    --low-memory   

Please visit https://docs.nvidia.com/clara/#parabricks for detailed documentation


[Parabricks Options Mesg]: Automatically generating ID prefix
[Parabricks Options Mesg]: Read group created for /home/yingja1227/germline_data/sample_1.fq.gz and
/home/yingja1227/germline_data/sample_2.fq.gz
[Parabricks Options Mesg]: @RG\tID:HK3TJBCX2.1\tLB:lib1\tPL:bar\tSM:sample\tPU:HK3TJBCX2.1


[Parabricks Options Mesg]: Checking argument compatibility
[Parabricks Options Mesg]: Set --bwa-options="-K #" to produce compatible pair-ended results with previous versions of
fq2bam or BWA MEM.
[Parabricks Options Mesg]: Read group created for /home/yingja1227/germline_data/sample_1.fq.gz and
/home/yingja1227/germline_data/sample_2.fq.gz
[Parabricks Options Mesg]: @RG\tID:HK3TJBCX2.1\tLB:lib1\tPL:bar\tSM:sample\tPU:HK3TJBCX2.1
[Parabricks Options Mesg]: Using --low-memory sets the number of streams in bwa mem to 1.
[PB Info 2025-Aug-29 13:47:27] ------------------------------------------------------------

[PB Info 2025-Aug-27 16:33:50] chr6:62342401	1.8	4777598	2605962
[PB Info 2025-Aug-27 16:34:00] chr6:161044801	2.0	5186472	2593236
[PB Info 2025-Aug-27 16:34:10] chr7:107764801	2.2	5705770	2633432
[PB Info 2025-Aug-27 16:34:20] chr8:62294401	2.3	6177174	2647360
[PB Info 2025-Aug-27 16:34:30] chr9:21998401	2.5	6609104	2643641
[PB Info 2025-Aug-27 16:34:40] chr10:29961601	2.7	7094248	2660343
[PB Info 2025-Aug-27 16:34:50] chr11:3681601	2.8	7592991	2679879
[PB Info 2025-Aug-27 16:35:00] chr11:122064001	3.0	8123867	2707955
[PB Info 2025-Aug-27 16:35:10] chr12:92136001	3.2	8551733	2700547
[PB Info 2025-Aug-27 16:35:20] chr13:102849601	3.3	9047105	2714131
[PB Info 2025-Aug-27 16:35:30] chr15:41577601	3.5	9582370	2737820
[PB Info 2025-Aug-27 16:35:40] chr16:56817601	3.7	10061794	2744125
[PB Info 2025-Aug-27 16:35:50] chr17:68524801	3.8	10464699	2729921
[PB Info 2025-Aug-27 16:36:00] chr19:32352001	4.0	10955058	2738764
[PB Info 2025-Aug-27 16:36:10] chr20:58627201	4.2	11338974	2721353
[PB Info

In [6]:
!ls germline_output

germline_out.bam      germline_out_chrs.txt  variants.vcf
germline_out.bam.bai  recal.table


### Run Germline Workflow: BWA-MEM + DeepVariant

Germline workflow with DeepVariant is also available using the below command.

In [9]:
!pbrun deepvariant_germline \
    --ref Ref/Homo_sapiens_assembly38.fasta \
    --in-fq germline_data/sample_1.fq.gz germline_data/sample_2.fq.gz \
    --out-bam germline_output/dvgermline_out.bam \
    --out-variants germline_output/variants_dv.vcf 
    #--low-memory    

Please visit https://docs.nvidia.com/clara/#parabricks for detailed documentation


[Parabricks Options Mesg]: Automatically generating ID prefix
[Parabricks Options Mesg]: Read group created for /home/yingja1227/germline_data/sample_data/Data/sample_1.fq.gz and
/home/yingja1227/germline_data/sample_data/Data/sample_2.fq.gz
[Parabricks Options Mesg]: @RG\tID:HK3TJBCX2.1\tLB:lib1\tPL:bar\tSM:sample\tPU:HK3TJBCX2.1


[Parabricks Options Mesg]: Checking argument compatibility
[Parabricks Options Mesg]: Read group created for /home/yingja1227/germline_data/sample_data/Data/sample_1.fq.gz and
/home/yingja1227/germline_data/sample_data/Data/sample_2.fq.gz
[Parabricks Options Mesg]: @RG\tID:HK3TJBCX2.1\tLB:lib1\tPL:bar\tSM:sample\tPU:HK3TJBCX2.1
[PB Info 2025-Aug-27 16:39:11] ------------------------------------------------------------------------------
[PB Info 2025-Aug-27 16:39:11] ||                 Parabricks accelerated Genomics Pipeline                 ||
[PB Info 2025-Aug-27 16:39:11] 

[PB Info 2025-Aug-27 16:45:06] ProgressMeter -	chr1:158292000	0.4
[PB Info 2025-Aug-27 16:45:12] ProgressMeter -	chr1:229843000	0.5
[PB Info 2025-Aug-27 16:45:18] ProgressMeter -	chr2:40154000	0.6
[PB Info 2025-Aug-27 16:45:24] ProgressMeter -	chr2:146693000	0.7
[PB Info 2025-Aug-27 16:45:30] ProgressMeter -	chr3:4036000	0.8
[PB Info 2025-Aug-27 16:45:36] ProgressMeter -	chr3:119975000	0.9
[PB Info 2025-Aug-27 16:45:42] ProgressMeter -	chr4:15711000	1.0
[PB Info 2025-Aug-27 16:45:48] ProgressMeter -	chr4:140268000	1.1
[PB Info 2025-Aug-27 16:45:54] ProgressMeter -	chr5:67945000	1.2
[PB Info 2025-Aug-27 16:46:00] ProgressMeter -	chr5:132578000	1.3
[PB Info 2025-Aug-27 16:46:06] ProgressMeter -	chr6:36353000	1.4
[PB Info 2025-Aug-27 16:46:12] ProgressMeter -	chr7:22983000	1.5
[PB Info 2025-Aug-27 16:46:18] ProgressMeter -	chr7:108547000	1.6
[PB Info 2025-Aug-27 16:46:24] ProgressMeter -	chr8:106980000	1.7
[PB Info 2025-Aug-27 16:46:30] ProgressMeter -	chr9:83624000	1.8
[PB Info 2025-Aug-